In [1]:
%alias go runoak -i sqlite:obo:go

In [2]:
go relationships --direction down -p RO:0002211 .desc//p=i 'signal transduction' > output/reg-pathway.tsv

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("output/reg-pathway.tsv", sep="\t")
df

,subject,subject_label,predicate,predicate_label,object,object_label
0,GO:0001959,regulation of cytokine-mediated signaling pathway,RO:0002211,regulates,GO:0019221,cytokine-mediated signaling pathway
1,GO:0002155,regulation of thyroid hormone mediated signali...,RO:0002211,regulates,GO:0002154,thyroid hormone mediated signaling pathway
2,GO:0003307,regulation of Wnt signaling pathway involved i...,RO:0002211,regulates,GO:0003306,Wnt signaling pathway involved in heart develo...
3,GO:0008277,regulation of G protein-coupled receptor signa...,RO:0002211,regulates,GO:0007186,G protein-coupled receptor signaling pathway
4,GO:0008589,regulation of smoothened signaling pathway,RO:0002211,regulates,GO:0007224,smoothened signaling pathway
...,...,...,...,...,...,...
246,GO:2001233,regulation of apoptotic signaling pathway,RO:0002211,regulates,GO:0097190,apoptotic signaling pathway
247,GO:2001236,regulation of extrinsic apoptotic signaling pa...,RO:0002211,regulates,GO:0097191,extrinsic apoptotic signaling pathway
248,GO:2001239,regulation of extrinsic apoptotic signaling pa...,RO:0002211,regulates,GO:0097192,extrinsic apoptotic signaling pathway in absen...
249,GO:2001242,regulation of intrinsic apoptotic signaling pa...,RO:0002211,regulates,GO:0097193,intrinsic apoptotic signaling pathway


In [5]:
from oaklib import get_adapter
from oaklib.datamodels.vocabulary import IS_A, PART_OF
from oaklib.datamodels.input_specification import InputSpecification

In [6]:
def gaf_adapter(src):
    spec = InputSpecification(ontology_resources={"go": {"selector": "sqlite:obo:go"}},
                              association_resources={"gaf":
                                                     {"selector": f"gaf:{src}"}})
    return get_adapter(spec)

goa_human = gaf_adapter("goa_human")
goa_human

SqlImplementation(resource=OntologyResource(slug='obo:go', directory=None, scheme='sqlite', format=None, url=None, readonly=False, provider=None, local=False, in_memory=False, data=None, implementation_class=<class 'oaklib.implementations.sqldb.sql_implementation.SqlImplementation'>, import_depth=None), strict=False, _multilingual=None, autosave=True, exclude_owl_top_and_bottom=True, ontology_metamodel_mapper=None, _converter=None, auto_relax_axioms=None, cache_lookups=False, functional_writer=None, _association_index=AssociationIndex(_connection=<sqlalchemy.pool.base._ConnectionFairy object at 0x7fabea70d8b0>, _session=<sqlalchemy.orm.session.Session object at 0x7fabea70d910>, _engine=Engine(sqlite:///:memory:)), normalizers=[], engine=Engine(sqlite:////Users/cjm/.data/oaklib/go.db), _session=None, _connection=None, _ontology_metadata_model=None, _prefix_map=None, _information_content_cache=None, _relationships_by_subject_index=None, max_items_for_in_clause=1000, can_store_association

In [7]:
amigo = get_adapter("amigo:NCBITaxon:9606")

In [8]:
def overlap(s1, s2):
    return len(set(s1).intersection(s2)), len(set(s1).difference(s2)), len(set(s2).difference(s1))

In [9]:
def genes_for_pair(adapter, t1, t2):
    assocs1 = adapter.associations(objects=[t1], object_closure_predicates=[IS_A, PART_OF])
    assocs2 = adapter.associations(objects=[t2], object_closure_predicates=[IS_A, PART_OF])
    genes1 = {a.subject for a in assocs1}
    genes2 = {a.subject for a in assocs2}
    return genes1, genes2

pairs = genes_for_pair(goa_human, "GO:0003307", "GO:0003306")

In [10]:
pd.Series(list(overlap(*pairs)))

0    1
1    7
2    3
dtype: int64

In [13]:
df[['common', 'left', 'right']] = df.apply(lambda x: pd.Series(list(overlap(*genes_for_pair(goa_human, x['subject'], x['object'])))), axis=1)
    

In [14]:
df

,subject,subject_label,predicate,predicate_label,object,object_label,common,left,right
0,GO:0001959,regulation of cytokine-mediated signaling pathway,RO:0002211,regulates,GO:0019221,cytokine-mediated signaling pathway,36,115,338
1,GO:0002155,regulation of thyroid hormone mediated signali...,RO:0002211,regulates,GO:0002154,thyroid hormone mediated signaling pathway,2,4,1
2,GO:0003307,regulation of Wnt signaling pathway involved i...,RO:0002211,regulates,GO:0003306,Wnt signaling pathway involved in heart develo...,1,7,3
3,GO:0008277,regulation of G protein-coupled receptor signa...,RO:0002211,regulates,GO:0007186,G protein-coupled receptor signaling pathway,66,70,1157
4,GO:0008589,regulation of smoothened signaling pathway,RO:0002211,regulates,GO:0007224,smoothened signaling pathway,29,59,60
...,...,...,...,...,...,...,...,...,...
246,GO:2001233,regulation of apoptotic signaling pathway,RO:0002211,regulates,GO:0097190,apoptotic signaling pathway,100,270,211
247,GO:2001236,regulation of extrinsic apoptotic signaling pa...,RO:0002211,regulates,GO:0097191,extrinsic apoptotic signaling pathway,40,114,71
248,GO:2001239,regulation of extrinsic apoptotic signaling pa...,RO:0002211,regulates,GO:0097192,extrinsic apoptotic signaling pathway in absen...,12,32,21
249,GO:2001242,regulation of intrinsic apoptotic signaling pa...,RO:0002211,regulates,GO:0097193,intrinsic apoptotic signaling pathway,31,136,131


In [16]:
df.query("common / (left + right) > 0.3")

,subject,subject_label,predicate,predicate_label,object,object_label,common,left,right
1,GO:0002155,regulation of thyroid hormone mediated signali...,RO:0002211,regulates,GO:0002154,thyroid hormone mediated signaling pathway,2,4,1
10,GO:0009966,regulation of signal transduction,RO:0002211,regulates,GO:0007165,signal transduction,1760,1198,3003
22,GO:0030111,regulation of Wnt signaling pathway,RO:0002211,regulates,GO:0016055,Wnt signaling pathway,159,168,123
119,GO:0090092,regulation of transmembrane receptor protein s...,RO:0002211,regulates,GO:0007178,transmembrane receptor protein serine/threonin...,106,166,95
123,GO:0106056,regulation of calcineurin-mediated signaling,RO:0002211,regulates,GO:0097720,calcineurin-mediated signaling,11,30,6
140,GO:1900234,regulation of Kit signaling pathway,RO:0002211,regulates,GO:0038109,Kit signaling pathway,1,1,2
170,GO:1902238,regulation of intrinsic apoptotic signaling pa...,RO:0002211,regulates,GO:1990127,intrinsic apoptotic signaling pathway in respo...,1,2,0
212,GO:2000040,regulation of planar cell polarity pathway inv...,RO:0002211,regulates,GO:0003402,planar cell polarity pathway involved in axis ...,2,0,4
231,GO:2000313,regulation of fibroblast growth factor recepto...,RO:0002211,regulates,GO:0060825,fibroblast growth factor receptor signaling pa...,1,1,0
236,GO:2000446,regulation of macrophage migration inhibitory ...,RO:0002211,regulates,GO:0035691,macrophage migration inhibitory factor signali...,1,2,0
